In [1]:
#Enter folder name where the images and dataset files exist
image_folder = '../../Images'
data_file_folder = '../annotator_data'
annotator = 'Arnab'
to_annotate_file = 'to_annotate_' + annotator + '.csv'
keypoints_file = 'KEYPOINTS_DATASET_' + annotator + '.csv'
status_file = 'STATUS_' + annotator + '.csv'

window_size = (1028,768)

In [2]:
import sys

sys.path.append('..')

In [3]:
import numpy as np
import pandas as pd
import cv2 as cv
import sys
import string
import os
from kp_package.annotation_tool import *

In [4]:
#Ordinarily, you will not need to change this
#file where keypoint annotations would be saved
data_file_path = os.path.join(data_file_folder, keypoints_file)
#file where all images that have been annotated will be saved
status_file_path = os.path.join(data_file_folder, status_file)
#file where list of files to be annotated will be provided by moderator
to_annotate_path = os.path.join(data_file_folder, to_annotate_file)


In [5]:
if(__name__ == "__main__"):
    
    if(os.path.exists(to_annotate_path)):
      #If list of images to be annotated has been provided
      tmp = pd.read_csv(to_annotate_path)
      file_names = tmp['img_id'].values
    else:
      #else take all images from the images folder
      file_names = os.listdir(image_folder)
    

    if(os.path.exists(data_file_path)):
      #If some annotations are already done, load them
      data = pd.read_csv(data_file_path, index_col = False)
    else:
      #Create a new empty dataframe to store annotations and save it to a file
      data = pd.DataFrame(columns = ['id', 'pid', 'type', 'x', 'y', 'attr', 'person', 'img_id', 'hidden'])
      data.to_csv(data_file_path, index = False)
    
    if(os.path.exists(status_file_path)):
      #If status file exists, load it
      completed = pd.read_csv(status_file_path, index_col = False)
    else:
      #Else create a new empty datasframe and save it to a file
      completed = pd.DataFrame(columns = ['file_name','annotator','success', 'sent_for_review', 'accepted'])
      completed.to_csv(status_file_path, index = False)
    
    #To skip already annotated images. Only considers those entries which were completed successfully 
    completed_files = completed[completed['success'] == True]
    completed_files = completed_files['file_name'].values
    
    
    for file_name in file_names:

      #To skip annotated images and non image files
      if(file_name  in completed_files) or (not (file_name.endswith('.png'))):
        continue

      file_path = os.path.join(image_folder, file_name)
      All_annotations = Final_Annotation()
    
      img = cv.imread(file_path)
      
      added = tool_GUI(file_name, All_annotations, img = img, window_size = window_size)
      
      if(not added):
            #If quit without saving annotations
            print('no annotations added on the current image. Expunging from the list')
            
            #Commented portion is most probably not reuqired
            '''
            new_file = pd.DataFrame({'file_name':[file_name], 'annotator':[annotator], 'success':[False], 'sent_for_review': [False], 'accepted':[False]})
            completed = completed.append(new_file, ignore_index = True)
      
            completed.to_csv(status_file_path, index = False)
             '''   
            
      
      else:
          a = All_annotations.annotations
          add = pd.DataFrame(a)

          data = data.append(add, ignore_index = True)

          #file operations begin. Ensure integrity
          new_file = pd.DataFrame({'file_name':[file_name], 'annotator':[annotator], 'success':[True], 'sent_for_review':[False], 'accepted':[False]})
          tmp_path = os.path.join(data_file_folder, 'tmp_keypoints_dataset.csv')
          data.to_csv(tmp_path, index = False)
          #data stored in temp file

          completed = completed.append(new_file, ignore_index = True)

          completed.to_csv(status_file_path, index = False)
          os.remove(data_file_path)
          os.rename(tmp_path,data_file_path)
          #File operations completed

          print("Image " + file_name + " completed")
      
      inp = input("Want to Continue with next image (y/n")
      
      if(inp == 'y'):
        continue
      else:
        break

    print(completed['file_name'].values)


Forehead Added with id 1
Nose Added with id 2
Image anika shade.png completed
Want to Continue with next image (y/nn
['anika shade.png']
